In [1]:
!pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client langchain-google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.4/719.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
      Successfully uninstalled google-auth-2.43.0
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.55.0
    Uninstalling google-genai-1.55.0:
      Successfully uninstalled google-genai-1.55.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 

In [2]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build



In [3]:
import os
os.listdir('/content')

['.config', 'sample_data']

In [4]:

GMAIL_SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
def authorize_gmail():
    creds = None

    # Check if token already exists
    if os.path.isfile('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', GMAIL_SCOPES
        )

    # If no valid credentials, start manual OAuth
    if not creds or not creds.valid:
        oauth_flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json',
            scopes=GMAIL_SCOPES,
            redirect_uri='urn:ietf:wg:oauth:2.0:oob'
        )

        auth_link, _ = oauth_flow.authorization_url(
            access_type='offline',
            include_granted_scopes='true'
        )

        print("\n OPEN THIS LINK IN YOUR BROWSER:\n")
        print(auth_link)

        auth_code = input("\n Enter the authorization code here: ").strip()

        oauth_flow.fetch_token(code=auth_code)
        creds = oauth_flow.credentials

        # Save token
        with open('token.json', 'w') as f:
            f.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)
    service = get_gmail_service()

In [7]:
gmail_service = authorize_gmail()
print("Gmail authorization successful")



 OPEN THIS LINK IN YOUR BROWSER:

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1077630780818-mt5uinqlt0v0dktbl4eb7m34tg858kjv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=tErmYTBy7g5nwAvDMCr01S7Uzd1Bo6&access_type=offline&include_granted_scopes=true

 Enter the authorization code here: 4/1ASc3gC0mnCOLzWvJXui8H4WoscnBz7t57m6h9f4JS-A265U4PQ1UuZ6ezMY
Gmail authorization successful


In [8]:
def read_inbox(service):
    results = service.users().messages().list(
        userId='me',
        labelIds=['INBOX'],
        maxResults=5
    ).execute()

    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
        return

    for msg in messages:
        msg_data = service.users().messages().get(
            userId='me',
            id=msg['id'],
            format='metadata',
            metadataHeaders=['From', 'Subject']
        ).execute()

        headers = msg_data['payload']['headers']
        email_data = {}

        for h in headers:
            email_data[h['name']] = h['value']

        print("\n📧 New Email")
        print("From:", email_data.get('From'))
        print("Subject:", email_data.get('Subject'))

In [9]:
service = authorize_gmail()
print("Gmail service ready")

Gmail service ready


In [10]:
read_inbox(service)


📧 New Email
From: Google <no-reply@accounts.google.com>
Subject: Security alert

📧 New Email
From: Exim Bank Recruitment <noreply@jobalertshub.com>
Subject: Exim Bank Management Trainee & Deputy Manager Job Notification

📧 New Email
From: Spotify <no-reply@spotify.com>
Subject: Get back into your Spotify account

📧 New Email
From: GitHub <no-reply@github.com>
Subject: Build AI agents with the new GitHub Copilot SDK

📧 New Email
From: Zora Mahali <zoramahali7@gmail.com>
Subject: Re: Group Meeting Invitation


In [12]:
from openai import OpenAI

client = OpenAI(api_key="Open_api_key")
for model in client.models.list():
    print(model.id)


Available OpenAI models:
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano
gpt-i

In [44]:
! pip install langchain langgraph langchain-openai python-dotenv

In [67]:
# Setting up LLM
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

def llm_call(prompt: str) -> str:
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
        config=types.GenerateContentConfig(
            # It forces the model to output raw JSON only
            response_mime_type="application/json"
        )
    )
    return response.text

# if __name__ == "__main__":
#     print(llm_call("Hello, Gemini!"))

In [72]:
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# Gmail scope
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None

    TOKEN_PATH = '/content/token.json'
    CREDS_PATH = '/content/credentials.json'

    # Load token if exists
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)

    # If no valid creds, login
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDS_PATH, SCOPES
            )
            creds = flow.run_local_server(port=0)

        # Save token
        with open(TOKEN_PATH, 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

# Create Gmail service
service = get_gmail_service()

# Test Gmail connection
profile = service.users().getProfile(userId='me').execute()
print("✅ Gmail connected:", profile["emailAddress"])


✅ Gmail connected: bhavyamahali@gmail.com


In [86]:
def load_llm():
    from dotenv import load_dotenv
    from langchain_google_genai import ChatGoogleGenerativeAI

    load_dotenv()
    return ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature =0)

In [99]:
def rule_based_triage(subject, body):
    text = (subject + " " + body).lower()

    # IGNORE only very clear spam / promos
    ignore_keywords = [
        "job", "vacancy", "notification", "alert",
        "spotify", "bank", "exim", "offer", "sale",
        "promotion", "webinar", "program", "academy",
        "github", "copilot"
    ]

    if any(word in text for word in ignore_keywords):
        return "IGNORE"

    # RESPOND to human / meeting / replies
    respond_keywords = [
        "connect", "meeting", "invite", "invitation",
        "group", "discussion", "call", "schedule",
        "re:", "fwd:", "follow"
    ]

    if any(word in text for word in respond_keywords):
        return "RESPOND"

    # DEFAULT: RESPOND (VERY IMPORTANT)
    return "RESPOND"


In [100]:
print(rule_based_triage("I want to connect", ""))
print(rule_based_triage("Group Meeting Invitation", ""))
print(rule_based_triage("Security alert", ""))


RESPOND
RESPOND
IGNORE


In [107]:
def build_email_prompt(user_prompt: str):
    from langchain_core.prompts import (
        ChatPromptTemplate,
        SystemMessagePromptTemplate,
        HumanMessagePromptTemplate
    )

    system_text = f"""
You are an AI email assistant.

User instructions:
{user_prompt}

IMPORTANT RULES:
- Output STRICT JSON only
- JSON must be valid and parseable
- Do NOT use markdown
- Do NOT add names or signatures
- Reply must be polite, concise, and human-like

JSON FORMAT:
{{
  "action": "reply" or "ignore",
  "reply": "plain email reply text only"
}}
"""

    system_msg = SystemMessagePromptTemplate.from_template(system_text)

    human_msg = HumanMessagePromptTemplate.from_template(
        "Subject: {subject}\nFrom: {sender}\nBody: {body}"
    )

    return ChatPromptTemplate.from_messages([
        system_msg,
        human_msg
    ])


In [88]:
import base64
from email.mime.text import MIMEText

def fetch_unread_emails(service, max_results=5):
    return service.users().messages().list(
        userId="me",
        labelIds=["UNREAD"],
        maxResults=max_results
    ).execute().get("messages", [])

def extract_email(service, msg_id):
    msg = service.users().messages().get(
        userId="me",
        id=msg_id,
        format="full"
    ).execute()

    headers = msg["payload"]["headers"]
    subject = sender = ""

    for h in headers:
        if h["name"] == "Subject":
            subject = h["value"]
        elif h["name"] == "From":
            sender = h["value"]

    body = ""
    payload = msg["payload"]

    if "parts" in payload:
        for part in payload["parts"]:
            if part["mimeType"] == "text/plain":
                data = part["body"].get("data")
                if data:
                    body = base64.urlsafe_b64decode(data).decode()
    else:
        data = payload["body"].get("data")
        if data:
            body = base64.urlsafe_b64decode(data).decode()

    return subject, sender, body, msg["threadId"]

import re

def clean_email_body(body: str) -> str:
    # Remove markdown JSON blocks
    body = re.sub(r"```json[\s\S]*?```", "", body, flags=re.IGNORECASE)

    # Remove quoted replies (Gmail style)
    body = re.split(r"\nOn .* wrote:", body)[0]

    # Remove lines starting with >
    body = "\n".join(
        line for line in body.splitlines()
        if not line.strip().startswith(">")
    )

    return body.strip()

def send_reply(service, to, subject, reply, thread_id):
    message = MIMEText(reply)
    message["to"] = to
    message["subject"] = "Re: " + subject

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

    service.users().messages().send(
        userId="me",
        body={"raw": raw, "threadId": thread_id}
    ).execute()

def mark_read(service, msg_id):
    service.users().messages().modify(
        userId="me",
        id=msg_id,
        body={"removeLabelIds": ["UNREAD"]}
    ).execute()

In [115]:
def generate_reply():
    return (
        "Thank you for reaching out. "
        "To help you further, could you please provide more details "
        "about your request?"
    )


In [113]:
def process_all_unread_emails(service):
    messages = fetch_recent_emails(service, max_results=10)

    print(f"\n📬 Total unread emails: {len(messages)}\n")

    for i, msg in enumerate(messages, start=1):
        sender, subject, body = extract_email_parts(msg)

        print("=" * 70)
        print(f"📧 Processing email {i}/{len(messages)}")
        print(f"From   : {sender}")
        print(f"Subject: {subject}")
        print("Body preview:")
        print(body[:300])
        print("=" * 70)

        decision = rule_based_triage(subject, body)

        if decision == "IGNORE":
            print("❌ Ignored\n")
            continue

        reply_text = generate_reply()

        print("\n🤖 AI Draft Reply")
        print("-" * 50)
        print(reply_text)
        print("-" * 50)

        create_draft_from_text(
            service=service,
            to=sender,
            subject=subject,
            reply_text=reply_text
        )

        # Optional: mark as read if you already have this logic
        print("✅ Reply sent & marked as read\n")


In [92]:
from email.mime.text import MIMEText
import base64

def create_draft_from_text(service, to, subject, reply_text):
    message = MIMEText(reply_text)
    message["to"] = to
    message["subject"] = f"Re: {subject}"

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

    service.users().drafts().create(
        userId="me",
        body={"message": {"raw": raw}}
    ).execute()


In [93]:
USER_PROMPT = """
Reply only to genuine emails.
Ignore promotions and job ads.
Be polite, concise, and human-like.
Do not add name or signature.
"""


In [118]:
process_all_unread_emails(service)



📬 Total unread emails: 10

📧 Processing email 1/10
From   : "IIT Kanpur & EICT Academy" <updates@simplilearnmailer.com>
Subject: Turn Data into Decisions with AI | Data Analytics & GenAI Program – E&ICT Academy, IIT Kanpur
Body preview:
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html dir="ltr" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns="http://www.w3.org/1999/xhtml" lang="en"><head><meta content="width=device-width, initial-scale=1" name="vi
❌ Ignored

📧 Processing email 2/10
From   : Podili Sri Sai Latha <invitations@linkedin.com>
Subject: I want to connect
Body preview:
Podili is waiting for your response
        
Hi Bhavya, I’d like to join your professional network

Podili Sri Sai Latha
Attended Vignan's Lara Institute of Technology and Science, Vadlamudi, Chebrolu Mandal, PIN-522213(CC-FE)
Guntur East
2 connections in common
See all connections in common

🤖 AI Draft Reply
-----------------